# Style transfer exploration

In [5]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

sys.path.append('../deep-latent-sequence-model/src')

In [6]:
model_path = '../deep-latent-sequence-model/outputs_yelp/yelp_wd0.0_wb0.0_ws0.0_an3_pool5_klw0.1_lr0.001_t0.01_lm_bt_hard_avglen/model.pt'
model = torch.load(model_path)

/home/przemyslaw/.local/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/przemyslaw/.local/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/przemyslaw/.local/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing th

In [7]:
model

Seq2Seq(
  (encoder): Encoder(
    (word_emb): Embedding(9653, 128, padding_idx=0)
    (layer): LSTM(128, 512, batch_first=True, dropout=0.3, bidirectional=True)
    (bridge): Linear(in_features=1024, out_features=512, bias=False)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (attention): MlpAttn(
      (dropout): Dropout(p=0.3, inplace=False)
      (w_trg): Linear(in_features=512, out_features=512, bias=True)
      (w_att): Linear(in_features=512, out_features=1, bias=True)
    )
    (ctx_to_readout): Linear(in_features=1536, out_features=512, bias=False)
    (readout): Linear(in_features=512, out_features=9653, bias=False)
    (word_emb): Embedding(9653, 128, padding_idx=0)
    (attr_emb): Embedding(2, 128, padding_idx=0)
    (layer): LSTMCell(1152, 512)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (enc_to_k): Linear(in_features=1024, out_features=512, bias=False)
  (noise): NoiseLayer()
  (LM0): LSTM_LM(
    (embed): Embedding(9653, 128, padding_id

This is a model we use for style transfer. It is encoder-decoder architecture.

In [21]:
def to_device(*args):
    device = torch.device('cuda')
    return [x.to(device) for x in args]

In [40]:
x = torch.tensor([[1, 2, 3]])
x_mask = torch.ones_like(x)
x_len = torch.tensor([1])
x, x_mask, x_len = to_device(x, x_mask, x_len)
x_pos_emb_idxs = None
y = x
y_mask = x_mask
y_len = x_len
y_pos_emb_idxs = None

x, x_mask, x_len = to_device(x, x_mask, x_len)

In [41]:
result = model(x, x_mask, x_len, x_pos_emb_idxs, y, y_mask, y_len, y_pos_emb_idxs, y, y_mask, y_len, y_pos_emb_idxs)

result: trans_logits, noise_logits, KL_loss, lm_length, trans_length

In [42]:
result

(tensor([[[3.6791, 3.0742, 3.2360,  ..., 3.0484, 2.1171, 3.4404],
          [1.1189, 1.1896, 0.6639,  ..., 1.6553, 0.5154, 0.7072]]],
        device='cuda:0', grad_fn=<TransposeBackward0>),
 None,
 tensor([[1.8127, 1.5167, 1.2207]], device='cuda:0', grad_fn=<DivBackward0>),
 5,
 4)

In [43]:
result[0].shape

torch.Size([1, 2, 9653])

In [45]:
result[2], result[2].shape

(tensor([[1.8127, 1.5167, 1.2207]], device='cuda:0', grad_fn=<DivBackward0>),
 torch.Size([1, 3]))